In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [3]:
spark = SparkSession.builder.appName("MostObscureSuperheroes").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/03 20:57:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
schema = StructType([ \
                     StructField("id", IntegerType(), True), \
                     StructField("name", StringType(), True)])

In [5]:
names = spark.read.schema(schema).option("sep", " ").csv("resources/Marvel-Names")

In [6]:
lines = spark.read.text("resources/Marvel-Graph")

In [9]:
connections = lines.withColumn("id", func.split(func.trim(func.col("value")), " ")[0]) \
    .withColumn("connections", func.size(func.split(func.trim(func.col("value")), " ")) - 1) \
        .groupBy("id").agg(func.sum("connections").alias("connections"))

In [19]:
minConnectionCount = connections.agg(func.min("connections")).first()[0]

In [20]:
minConnections = connections.filter(func.col("connections") == minConnectionCount)

In [21]:
minConnectionsWithNames = minConnections.join(names, "id")

In [22]:
print("The following charaters have only " + str(minConnectionCount) + " connection(s):")

The following charaters have only 0 connection(s):


In [23]:
minConnectionsWithNames.select("name").show()

+--------------------+
|                name|
+--------------------+
|        BERSERKER II|
|              BLARE/|
|MARVEL BOY II/MARTIN|
|MARVEL BOY/MARTIN BU|
|      GIURESCU, RADU|
|       CLUMSY FOULUP|
|              FENRIS|
|              RANDAK|
|           SHARKSKIN|
|     CALLAHAN, DANNY|
|         DEATHCHARGE|
|                RUNE|
|         SEA LEOPARD|
|         RED WOLF II|
|              ZANTOR|
|JOHNSON, LYNDON BAIN|
|          LUNATIK II|
|                KULL|
|GERVASE, LADY ALYSSA|
+--------------------+

